    Import the required packages and collect responses regarding which data is available for use. For now, that data has been hardcoded.

In [1]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime

# personName = raw_input('Enter your full name: ')
personName = "Ayushya Chitransh"
# fbData = raw_input('Do you have Facebook data to parse through (y/n)?')
fbData = "n"
# googleData = raw_input('Do you have Google Hangouts data to parse through (y/n)?')
googleData  ="y"
# linkedInData = raw_input('Do you have LinkedIn data to parse through (y/n)?')
linkedInData = "y"
# whatsAppData = raw_input('Do you have whatsAppData to parse through (y/n)?')
whatsAppData = "n"
print "Following information has been collected:\n"
print "Name: \t\t\t\t", personName
print "Facebook Data available:\t", fbData
print "Google Data available:\t\t", googleData
print "LinkedIn Data available:\t", linkedInData
print "Whatsapp Data available:\t", whatsAppData

Following information has been collected:

Name: 				Ayushya Chitransh
Facebook Data available:	n
Google Data available:		y
LinkedIn Data available:	y
Whatsapp Data available:	n


    Declare functions to parse and clean various data sources.
    - Whatsapp Data parser
    - Hangouts data parser
    - Linkedin Data parser
    - Facebook data parser

In [2]:
def getWhatsAppData():
        df = pd.read_csv('whatsapp_chats.csv')
        responseDictionary = dict()
        receivedMessages = df[df['From'] != personName]
        sentMessages = df[df['From'] == personName]
        combined = pd.concat([sentMessages, receivedMessages])
        otherPersonsMessage, myMessage = "",""
        firstMessage = True
        for index, row in combined.iterrows():
            if (row['From'] != personName):
                if myMessage and otherPersonsMessage:
                    otherPersonsMessage = cleanMessage(otherPersonsMessage)
                    myMessage = cleanMessage(myMessage)
                    responseDictionary[otherPersonsMessage.rstrip()] = myMessage.rstrip()
                    otherPersonsMessage, myMessage = "",""
                otherPersonsMessage = otherPersonsMessage + str(row['Content']) + " "
            else:
                if (firstMessage):
                    firstMessage = False
                    # Don't include if I am the person initiating the convo
                    continue
                myMessage = myMessage + str(row['Content']) + " "
        return responseDictionary

In [3]:
def getWhatsAppData():
        df = pd.read_csv('whatsapp_chats.csv')
        responseDictionary = dict()
        receivedMessages = df[df['From'] != personName]
        sentMessages = df[df['From'] == personName]
        combined = pd.concat([sentMessages, receivedMessages])
        otherPersonsMessage, myMessage = "",""
        firstMessage = True
        for index, row in combined.iterrows():
            if (row['From'] != personName):
                if myMessage and otherPersonsMessage:
                    otherPersonsMessage = cleanMessage(otherPersonsMessage)
                    myMessage = cleanMessage(myMessage)
                    responseDictionary[otherPersonsMessage.rstrip()] = myMessage.rstrip()
                    otherPersonsMessage, myMessage = "",""
                otherPersonsMessage = otherPersonsMessage + str(row['Content']) + " "
            else:
                if (firstMessage):
                    firstMessage = False
                    # Don't include if I am the person initiating the convo
                    continue
                myMessage = myMessage + str(row['Content']) + " "
        return responseDictionary

In [4]:
def getGoogleHangoutsData(location):
    # Putting all the file names in a list
    allFiles = []
    # Edit these file and directory names if you have them saved somewhere else
    for filename in os.listdir(location):
        if filename.endswith(".txt"): 
            allFiles.append(location + "/" + filename)

    responseDictionary = dict() # The key is the other person's message, and the value is my response
    # Going through each file, and recording everyone's messages to me, and my responses
    for currentFile in allFiles:
        myMessage, otherPersonsMessage, currentSpeaker = "","",""
        openedFile = open(currentFile, 'r') 
        allLines = openedFile.readlines()
        for index,lines in enumerate(allLines):
            # The sender's name is separated by < and >
            leftBracket = lines.find('<')
            rightBracket = lines.find('>')
            
            # Find messages that I sent
            if (lines[leftBracket+1:rightBracket] == personName):
                if not myMessage:
                    # Want to find the first message that I send (if I send multiple in a row)
                    startMessageIndex = index - 1
                myMessage += lines[rightBracket+1:]
            
            elif myMessage:
                # Now go and see what message the other person sent by looking at previous messages
                for counter in range(startMessageIndex, 0, -1):
                    currentLine = allLines[counter]
                    # In case the message above isn't in the right format
                    if (currentLine.find('<') < 0 or currentLine.find('>') < 0):
                        myMessage, otherPersonsMessage, currentSpeaker = "","",""
                        break
                    if not currentSpeaker:
                        # The first speaker not named me
                        currentSpeaker = currentLine[currentLine.find('<')+1:currentLine.find('>')]
                    elif (currentSpeaker != currentLine[currentLine.find('<')+1:currentLine.find('>')]):
                        # A different person started speaking, so now I know that the first person's message is done
                        otherPersonsMessage = cleanMessage(otherPersonsMessage)
                        myMessage = cleanMessage(myMessage)
                        responseDictionary[otherPersonsMessage] = myMessage
                        break
                    otherPersonsMessage = currentLine[currentLine.find('>')+1:] + otherPersonsMessage
                myMessage, otherPersonsMessage, currentSpeaker = "","",""
    return responseDictionary

In [5]:
def getFacebookData():
    responseDictionary = dict()
    fbFile = open('fbMessages.txt', 'r') 
    allLines = fbFile.readlines()
    myMessage, otherPersonsMessage, currentSpeaker = "","",""
    for index,lines in enumerate(allLines):
        rightBracket = lines.find(']') + 2
        justMessage = lines[rightBracket:]
        colon = justMessage.find(':')
        # Find messages that I sent
        if (justMessage[:colon] == personName):
            if not myMessage:
                # Want to find the first message that I send (if I send multiple in a row)
                startMessageIndex = index - 1
            myMessage += justMessage[colon+2:]
            
        elif myMessage:
            # Now go and see what message the other person sent by looking at previous messages
            for counter in range(startMessageIndex, 0, -1):
                currentLine = allLines[counter]
                rightBracket = currentLine.find(']') + 2
                justMessage = currentLine[rightBracket:]
                colon = justMessage.find(':')
                if not currentSpeaker:
                    # The first speaker not named me
                    currentSpeaker = justMessage[:colon]
                elif (currentSpeaker != justMessage[:colon] and otherPersonsMessage):
                    # A different person started speaking, so now I know that the first person's message is done
                    otherPersonsMessage = cleanMessage(otherPersonsMessage)
                    myMessage = cleanMessage(myMessage)
                    responseDictionary[otherPersonsMessage] = myMessage
                    break
                otherPersonsMessage = justMessage[colon+2:] + otherPersonsMessage
            myMessage, otherPersonsMessage, currentSpeaker = "","",""    
    return responseDictionary

In [6]:
def getLinkedInData(file_name):
    df = pd.read_csv(file_name)
    dateTimeConverter = lambda x: datetime.strptime(x,'%x, %I:%M %p')
    responseDictionary = dict()
    peopleContacted = df['From'].unique().tolist()
    for person in peopleContacted:
        receivedMessages = df[df['From'] == person]
        sentMessages = df[df['To'] == person]
        if (len(sentMessages) == 0 or len(receivedMessages) == 0):
            # There was no actual conversation
            continue
        combined = pd.concat([sentMessages, receivedMessages])
        combined['Date'] = combined['Date'].apply(dateTimeConverter)
        combined = combined.sort_values(['Date'])
        otherPersonsMessage, myMessage = "",""
        firstMessage = True
        for index, row in combined.iterrows():
            if (row['From'] != personName):
                if myMessage and otherPersonsMessage:
                    otherPersonsMessage = cleanMessage(otherPersonsMessage)
                    myMessage = cleanMessage(myMessage)
                    responseDictionary[otherPersonsMessage.rstrip()] = myMessage.rstrip()
                    otherPersonsMessage, myMessage = "",""
                if(type(row['Content']) != float):
                    otherPersonsMessage = otherPersonsMessage + row['Content'] + " "
            else:
                if (firstMessage):
                    firstMessage = False
                    # Don't include if I am the person initiating the convo
                    continue
                myMessage = myMessage + str(row['Content']) + " "
    return responseDictionary

    A helper utility to clean messages and remove newlines, non-ascii characters, punctuations, maybe more to come.

In [7]:
def cleanMessage(message):
    # Remove new lines within message
    cleanedMessage = message.replace('\n',' ').lower()
    # Deal with some weird tokens
    cleanedMessage = cleanedMessage.decode('utf8').encode('ascii', errors='ignore')
    # Remove punctuation
    cleanedMessage = re.sub('([.,!?])','', cleanedMessage)
    # Remove multiple spaces in message
    cleanedMessage = re.sub(' +',' ', cleanedMessage)
    return cleanedMessage

    Clean each data source one by one.

In [8]:
combinedDictionary = {}
if (googleData == 'y'):
    print 'Getting Google Hangout Data'
    combinedDictionary.update(getGoogleHangoutsData('dirtyData/hangouts/dataType1'))
    combinedDictionary.update(getGoogleHangoutsData('dirtyData/hangouts/dataType2'))

Getting Google Hangout Data


In [9]:
if (fbData == 'y'):
    print 'Getting Facebook Data'
    combinedDictionary.update(getFacebookData())

In [10]:
if (linkedInData == 'n'):
    print 'Getting LinkedIn Data'
    combinedDictionary.update(getLinkedInData('dirtyData/linkedIn/Messages.csv'))

In [11]:
if (whatsAppData == 'y'):
        print 'Getting whatsApp Data'
        combinedDictionary.update(getWhatsAppData())

In [12]:
print 'Total len of dictionary', len(combinedDictionary)
print 'Saving conversation data dictionary'
np.save('conversationDictionary.npy', combinedDictionary)

Total len of dictionary 4253
Saving conversation data dictionary
